In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import cartopy as cp
from matplotlib.patches import FancyArrowPatch
from scipy.ndimage import map_coordinates
import metpy.calc as mpcalc
from geographiclib.geodesic import Geodesic
from utils_datetime import *
from utils_filter import *

In [ ]:
grid_outlooks = xr.open_dataset('data/outlooks/grid_outlooks.nc')
grid_outlooks = grid_outlooks.sel(time = grid_outlooks['time'] >= '200203300000')

In [ ]:
# creates new displacements dataset by keeping data (get rid of shifts and div) 
# but changing x and y coords so that (0, 0) is at max or weighted center of outlook (or pph?). Since highest risk is seen as "center" of what will happen, even if weaker storms are focused to one direction
# then calculate things like shifts of quadrents

# note: grids will not quite be aligned when each is centered over a different point, but probably close enough


# finding x and y to make center for each date
grouped = grid_outlooks['prob'].groupby('time')

# Step 1: Find all points with the maximum prob for each day and compute mean coordinates
def find_mean_coords(group):
    max_prob = group.max()  # Maximum value in the group
    if max_prob == 0:
        mean_x = group['x'].mean().item()
        mean_y = group['y'].mean().item()
    else:
        # Select all points with prob == max_prob
        max_points = group.where(group == max_prob, drop=True)
        # Compute the mean of x and y
        mean_x = max_points['x'].mean().item()
        mean_y = max_points['y'].mean().item()
    # Round to nearest integers
    nearest_x = round(mean_x)
    nearest_y = round(mean_y)
    # Return as a Dataset
    return xr.Dataset({'nearest_x': nearest_x, 'nearest_y': nearest_y})

# Apply the function to each group
center_coords = grouped.map(find_mean_coords)
center_coords

del grid_outlooks, grouped

<xarray.Dataset>
Dimensions:    (time: 7947)
Coordinates:
  * time       (time) object '200203300000' '200203310000' ... '202312310000'
Data variables:
    nearest_x  (time) int32 50 65 55 56 67 55 70 44 ... 46 46 46 46 46 46 46 46
    nearest_y  (time) int32 18 22 32 24 24 21 14 22 ... 32 32 32 32 32 32 32 32

In [19]:
# Extract nearest_x and nearest_y
nearest_x = center_coords['nearest_x']
nearest_y = center_coords['nearest_y']


# Re-center displacements
def recenter_displacements(displacements, nearest_x, nearest_y):
    # Shift x and y coordinates based on nearest_x, nearest_y for each time
    new_x = displacements['x'] - nearest_x
    new_y = displacements['y'] - nearest_y
    
    # Assign shifted coordinates
    displacements = displacements.assign_coords({
        'x': new_x,
        'y': new_y
    })
    
    # Update lat and lon to depend on time, x, and y
    
    
    return displacements
# Initialize an empty list to store the recentered displacements



for dataset_location in ['data/pph/labelled_pph', 'data/displacement/displacements', 'data/outlooks/grid_outlooks', ]:
    print(dataset_location)
    recentered_list = []
    ds = xr.open_dataset(dataset_location + '.nc')
    ds = ds.sel(time = ds['time'] >= '200203300000')

    # Loop over each time step and apply recenter_displacements
    for t in range(len(ds.time)):
        # Get the current time slice of displacements
        displacement_slice = ds.isel(time=t)

        # Get the corresponding nearest_x and nearest_y for this time step
        nearest_x_t = nearest_x.isel(time=t)
        nearest_y_t = nearest_y.isel(time=t)

        

        # Apply the recentering function to the current time slice
        recentered_t = recenter_displacements(displacement_slice, nearest_x_t, nearest_y_t)

        # expanding needs to be done differently

        # Append the recentered displacements for this time slice
        recentered_list.append(recentered_t)
    print('combining')
    # Combine the recentered displacements back into a single xarray object
    recentered = xr.concat(recentered_list, dim='time')
    # Ensure proper ordering of dimensions
    #recentered = recentered.transpose('time', 'y', 'x', 'hazard')
    recentered.to_netcdf(dataset_location + '_recentered.nc')
    del recentered, recentered_list


data/displacement/displacements
combining
data/outlooks/grid_outlooks
combining


In [20]:
# open pph, outlook, and displacements
# do something like how shifts and divergence are calculated but in each quadrent
pph_recentered = xr.open_dataset('data/pph/labelled_pph_recentered.nc')
outlooks_recentered = xr.open_dataset('data/outlooks/grid_outlooks_recentered.nc')
displacements_recentered = xr.open_dataset('data/displacement/displacements_recentered.nc')


In [26]:
displacements_recentered

<xarray.Dataset>
Dimensions:    (x: 152, y: 99, hazard: 4, time: 7947)
Coordinates:
  * x          (x) float64 -80.0 -79.0 -78.0 -77.0 -76.0 ... 68.0 69.0 70.0 71.0
  * y          (y) float64 -44.0 -43.0 -42.0 -41.0 -40.0 ... 51.0 52.0 53.0 54.0
  * hazard     (hazard) object 'Wind' 'Hail' 'Tornado' 'All Hazard'
  * time       (time) object '200203300000' '200203310000' ... '202312310000'
Data variables:
    lat        (time, y, x) float64 ...
    lon        (time, y, x) float64 ...
    x_flow     (time, y, x, hazard) float64 ...
    y_flow     (time, y, x, hazard) float64 ...
    end_lon    (time, y, x, hazard) float64 ...
    end_lat    (time, y, x, hazard) float64 ...
    e_flow     (time, y, x, hazard) float64 ...
    n_flow     (time, y, x, hazard) float64 ...
    e_shift    (time, hazard) float64 ...
    n_shift    (time, hazard) float64 ...
    total_div  (time, hazard) float64 ...
Attributes:
    description:  Displacements from gridded day 1 probabilistic outlooks to ...
    grid:         80-km NCEP 211